In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from sklearn.utils import resample

In [2]:
# load tatanic dataset
def load_titanic():
    titanic_data = pd.read_csv('titanic/train_after.csv')
    print(titanic_data.head())
    y = titanic_data['Survived']
    print(y)

    # feature_names = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Has_Cabin', 'FamilySize', 'Title']
    feature_names = ['Pclass', 'Sex', 'Age']
    x = titanic_data[feature_names]
    print(x)
    
    return x, y, feature_names

In [3]:
# load dataset
def load_dataset(name, feature_num, discret_cat=5):
    datafile = './{}/{}.csv'.format(name, name)
    data_pd = pd.read_csv(datafile)
    feature_names = []
    for i in range (1,feature_num+1):
        feature_name = 'f'+str(i)
        if discret_cat > 0: # need discretization
            data_pd[feature_name+'_c'] = pd.cut(data_pd[feature_name], discret_cat, labels = list(range(discret_cat)))
            feature_name += '_c'
        feature_names.append(feature_name)
    data_pd.head()
    y = data_pd['y']
    x = data_pd[feature_names]
    return x, y, feature_names

In [4]:
# for calculating mutual information
from collections import Counter
def our_entropy(labels): # H(A)
    pro_dict = Counter(labels) #计数
    s = sum(pro_dict.values())#总数
    probs = np.array([i/s for i in pro_dict.values()])#概率
    return - probs.dot(np.log(probs))
def MI_(s1,s2):# 互信息
    s_s_1=["%s%s"%(i,j) for i,j in zip(s1,s2)]
    MI_1=our_entropy(s1)+our_entropy(s2)-our_entropy(s_s_1)
    return MI_1
def N_MI(s1,s2): # 标准化互信息
    MI_1 = MI_(s1,s2)
    NMI_1 = MI_1/(our_entropy(s1)*our_entropy(s2))**0.5
    return NMI_1

In [5]:
# get all the permutations of the features and then calculate conditional mutual information regarding Y
import itertools

#x, y, feature_names = load_titanic()
#x, y, feature_names = load_dataset('wine', 13) 
#x, y, feature_names = load_dataset('parkinsons', 22)
x, y, feature_names = load_dataset('breast', 30)
#x, y, feature_names = load_dataset('ionosphere', 34)
#x, y, feature_names = load_dataset('landsat', 36)
#x, y, feature_names = load_dataset('spect', 22, 0)
#x, y, feature_names = load_dataset('congress', 16, 0)
#x, y, feature_names = load_dataset('winequality-red', 11)
#x, y, feature_names = load_dataset('winequality-white', 11)


X_train,X_test,Y_train,Y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# all_feature_permutations = list(itertools.permutations(feature_names)) # time-consuming if feature number is large

contribution = {}
for feature_name in feature_names:
    contribution[feature_name] = []

Y_value_list = Y_train.values.tolist()

# for each_permutation in all_feature_permutations:
random_permutation_times = 20000
for i in range(random_permutation_times): # random sample permutations for 10000 times
    each_permutation = np.random.permutation(feature_names)
    current_feature_set = []
    current_MI = 0
    for feature_name in each_permutation:
        current_feature_set.append(feature_name)
        x_new = X_train[current_feature_set]
        new_MI = MI_(Y_value_list, list(x_new.itertuples(index=False)))
        contr = new_MI - current_MI # conditional CMI of the current feature in the specific permutation
        contribution[feature_name].append(contr) # add the CMI together in all the permutations
        current_MI = new_MI
    i += 1
    
    if i%100 == 0:
        features_values = [0]*len(feature_names)
        for feature_i in range(len(feature_names)):
            feature_name = feature_names[feature_i]
            features_values[feature_i] = np.mean(contribution[feature_name])
        features_values = features_values/np.sum(features_values)
       
        # calculate bootstrapped deviation
#         bootstrap_times = 1000
#         entropy_bootstrap = []
#         for ii in range(bootstrap_times):
#             bootstrapSamples = resample(range(i), n_samples=i, replace=1)
#             features_values_bootstrap = [0]*len(feature_names)
#             for feature_i in range(len(feature_names)):
#                 feature_name = feature_names[feature_i]
#                 for jj in bootstrapSamples:
#                     features_values_bootstrap[feature_i] += contribution[feature_name][jj]/i
#             entropy_bootstrap.append(entropy(features_values_bootstrap/np.sum(features_values_bootstrap)))
        
        print(i, entropy(features_values))#, np.std(entropy_bootstrap))

100 3.1923097025750775
200 3.190109246652498
300 3.189408104435425
400 3.189781947214143
500 3.1975124175566902
600 3.1997940940198535
700 3.1986157583021053
800 3.198811184392566
900 3.1965170324262737
1000 3.1930253229634817
1100 3.1900495479874498
1200 3.1885623694814154
1300 3.187078809786947
1400 3.1879976983436027
1500 3.1880963784078125
1600 3.186031503189776
1700 3.185874006599152
1800 3.1857000102939543
1900 3.1858573521414812
2000 3.186087290684896
2100 3.1866159676565706
2200 3.1861829681239
2300 3.1863334465510427
2400 3.187088433631875
2500 3.1862984354382125
2600 3.1859395025300334
2700 3.1848861287476122
2800 3.183923161526981
2900 3.1837575538753144
3000 3.1828916948408876
3100 3.182605802737027
3200 3.1831768661002524
3300 3.18313618795672
3400 3.1825048650306536
3500 3.18256397855283
3600 3.1822862376392336
3700 3.183518954839908
3800 3.182891734314046
3900 3.183418108026832
4000 3.183217935453146
4100 3.1837020520213484
4200 3.18390749628428
4300 3.1842004593948188
4